In [ ]:
!pip install transformers

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load a pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # Can choose other variations like "gpt2-medium" or "gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
#Format dataset as a text file with each question-answer pair on a separate line.
dataset = TextDataset( #Might need to convert csv file into txt file before doing this
    tokenizer=tokenizer,
    file_path="/content/drive/My Drive/dialogue_pairs.csv",  # Replace with the path to your dataset file
    block_size=128,  # Adjust this according to your dataset
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Prepare data for training
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",  # Output directory for the fine-tuned model
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    save_steps=1000,
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_steps=100,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
# Fine-tune the model
for epoch in range(training_args.num_train_epochs):
    for step, batch in enumerate(trainer.get_train_dataloader()):
        trainer.train()

        # Print current timestep and epoch
        if step % training_args.logging_steps == 0:
            print(f"Epoch [{epoch+1}/{training_args.num_train_epochs}] - Step [{step}]")


# Save the fine-tuned model
trainer.save_model()

Step,Training Loss,Validation Loss
